In [47]:
import warnings
warnings.filterwarnings('ignore')

# 앙상블(Ensenble)
- 여러개의 분류모델을 조합해서 더 나은 성능을 내는 방법
- Decision Tree 모델을 증가시켜 나온 랜덤포레스트가 대표적임

---
# 랜덤포레스트(Random Forest)
- 부트스트랩 샘플을 사용합니다. 부트스트랩 샘플링은 중복을 허용하는 샘플링 방법입니다.

In [48]:
import pandas as pd
wine = pd.read_csv("../Data/wine.csv")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [49]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [50]:
# 전체 셋 중에 훈련과 테스트 셋을 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [51]:
# 랜덤포레스트 모델 구현
import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
# n_jobs=-1 가용할수 있는 모든 cpu를 동원한다는 뜻
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores =cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.35243583, 0.47553301, 0.41016293, 0.46193194, 0.32635593]),
 'score_time': array([0.02835608, 0.03057885, 0.02532506, 0.02725601, 0.02410913]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [52]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [53]:
# 주요 Feature
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [54]:
# 부트스트랩 결정시 남은 샘플(oob : out of back)로도 특성을 구분 할 수 있다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


---
# Extra Tree
- 기본적으로 100개의 트리를 사용
- 노드 분할시 특성의 제곱근의 갯수를 사용
- 특성의 선택을 랜덤하게 선택
- 특성의 선택을 랜덤하게 하므로 속도는 랜덤포레스트보다 빠름


In [55]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores =cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [56]:
# 주요 Feature
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


---
# Gradient Boosting(그래디언트 부스팅)
- 가장 유명한 알고리즘 중 하나이다.
- 경사하강법처럼 손실함수를 사용.round
- 손실함수를 보고 트리를 추가하여 최적의 값 도출하는 방법
- Decision Tree Regressor를 사용하여 손실함수를 계산하고 이를 계속 낮추기 위해 트리를 추가하는 구조
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(기본:0.1)을 사용한다.
- max_depth를 3으로 제어하여 깊이가 낮으므로 과대적합 방지
- 단점을 손실함수를 보고 트리를 추가하면서 진행하는 모델이므로 병렬처리(n_jobs)를 할 수 없다.


In [57]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
# scores
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [58]:
# 기본값을 변경 : tree를 500개, learning rate를 0.2로
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
# scores
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [59]:
# 주요 Feature
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


---
# 히스토그램 기반 그래디언트 부스팅(Histogram gradient boosting)
- 훈련데이터를 256개의 구간으로 나누어서 훈련시키는 방법
- 특성의 범위가 제한되어 빠른 속도를 제공
- 제한된 구간으로 과대적합을 방지한다
- 아직은 실험단계인 모델

In [65]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
# scores
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [76]:
# 주요 Feature
hgb.fit(train_input, train_target)
print(hgb.feature_importances_)

AttributeError: 'HistGradientBoostingClassifier' object has no attribute 'feature_importances_'

# * 번외 기능 *
### Permutaion Importance(치환 중요도)
- 각 특성(Feature)별 샘플을 섞어서 계산을 한후에 원래 sample들과의 차이를 계산해서<br> 차이가 많이나는 Feature가 중요하다는 판단.

In [67]:
# Train 경우 확인
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


> Sugar인 경우 Sample을 무작위로 섞우면 23 % 만큼 정확도가 떨어진다.   
> 다른 특성은 Sample을 무작위로 섞어도 별 차이가 없다. 따로 주요 특성이 아니다.

In [68]:
# Test인 경우 (실전 투입시 예상)

result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)


[0.05969231 0.20238462 0.049     ]


---
# XGBoost
- kaggle에서 많이 사용

In [69]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [70]:
from xgboost import XGBClassifier

In [71]:
xgb = XGBClassifier(
    tree_method='hist', # 히스토그램 그레디언트 부스팅
    random_state = 42,
    use_label_encoder= False,
    eval_metric ="logloss" # 로지스틱 손실 함수
)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
# scores
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


---
# LightGBM


In [72]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [73]:
from lightgbm import LGBMClassifier


In [74]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
# scores
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


---
# 앙상블 정리
- 앙상블 학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중의 하나 입니다.

### 랜덤 포레스트
: 부트스트랩 사용, 대표 앙상블 학습 알고리즘   
### 엑스트라 트리
: 결정트리의 노드를 랜덤하게 분할함   
### 그레디언트 부스팅
: 결정트리의 손실을 보완하는 식으로 얕은 결정트리를 연속하여 추가함  
### 히스토그램기반 그레디언트 부스팅
: 훈련데이터를 256개의 정수 구간으로 나누어 빠르고 높은 성능을 냄   

### 기타
- XGBoost
- LightGBM